In [ ]:
!sudo apt-get update -y
!sudo apt-get install python3.8 python3.8-distutils python3.8-dev -y

!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!sudo update-alternatives --set python3 /usr/bin/python3.8
!wget https://bootstrap.pypa.io/pip/3.8/get-pip.py -O get-pip.py
!python3.8 get-pip.py

In [ ]:
!python --version

In [ ]:
# clone SadTalker
!git clone https://github.com/OpenTalker/SadTalker.git
%cd SadTalker

# install dependencies
!python3.8 -m pip install --upgrade pip setuptools wheel
!python3.8 -m pip install -r requirements.txt

# download pretrained checkpoints
!bash scripts/download_models.sh

In [ ]:
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113

In [ ]:
import sys
!{sys.executable} -m pip install flask pyngrok --ignore-installed blinker

In [ ]:
import sys, importlib, os
print("sys.executable:", sys.executable)
print("python version:", sys.version)
!{sys.executable} -m pip show pyngrok
!{sys.executable} -m pip list | grep pyngrok || true
print("find_spec:", importlib.util.find_spec("pyngrok"))

In [ ]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.8/dist-packages")
from pyngrok import ngrok
print("ok", ngrok)

In [ ]:
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok
import subprocess
import uuid
import shutil
import base64
import io

app = Flask(__name__)

@app.route('/lipsync', methods=['POST'])
def create_lipsync():
    try:
        # Decode base64 files
        data = request.json
        image_data = base64.b64decode(data['image'])
        audio_data = base64.b64decode(data['audio'])

        # Save files
        session_id = str(uuid.uuid4())
        temp_dir = f"/content/temp_{session_id}"
        os.makedirs(temp_dir, exist_ok=True)

        image_path = f"{temp_dir}/source.jpg"
        audio_path = f"{temp_dir}/audio.wav"
        results_dir = f"{temp_dir}/results"

        with open(image_path, 'wb') as f:
            f.write(image_data)
        with open(audio_path, 'wb') as f:
            f.write(audio_data)

        # Run SadTalker
        cmd = [
            'python', 'inference.py',
            '--driven_audio', audio_path,
            '--source_image', image_path,
            '--result_dir', results_dir,
            '--still',
            '--preprocess', 'crop',
            '--enhancer', 'gfpgan'
        ]

        subprocess.run(cmd, cwd='/content/SadTalker', check=True)

        # Find output video
        for root, dirs, files in os.walk(results_dir):
            for file in files:
                if file.endswith('.mp4'):
                    output_path = os.path.join(root, file)
                    with open(output_path, 'rb') as f:
                        video_data = base64.b64encode(f.read()).decode()

                    # Cleanup
                    shutil.rmtree(temp_dir)

                    return jsonify({
                        'success': True,
                        'video': video_data
                    })

        return jsonify({'success': False, 'error': 'No output generated'}), 500

    except Exception as e:
        return jsonify({'success': False, 'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy', 'service': 'sadtalker-colab'})

In [ ]:
from pyngrok import ngrok
import json

# Set your ngrok auth token (get from ngrok.com)
ngrok.set_auth_token("2zy1iAMFuUNxSNf8ZF1F0dYJsJO_3FjsdqffVjGAbRz7Ga1oe")  # Optional but recommended

# Start tunnel
public_url = ngrok.connect(8003)
print(f"Public URL: {public_url}")
print(f"API Endpoint: {public_url}/lipsync")

# Save URL to file for easy access
with open('/content/api_url.txt', 'w') as f:
    f.write(str(public_url))

In [ ]:
app.run(port=8003)